In [1]:
import mediapipe as mp
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import time as t
from djitellopy import Tello
import threading

In [2]:
path = r"C:\Users\Luis\OneDrive\Desktop\Maturaarbeit\codingMA\Mediapipe\model1.h5"

model = tf.keras.models.load_model(path)

tello_img = r"C:\Users\Luis\OneDrive\Desktop\Maturaarbeit\Tello\Tello_pictures\ "
num_pic = 0
cam_res = (640, 480)

signs = ["right", "left", "up", "down", "fist", "hand", "V"]

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.05)
drawingModule = mp.solutions.drawing_utils

def get_sign(frame):
    results = hands.process(frame)
        
    if results.multi_hand_landmarks: 
        points = []
        for handLandmarks in results.multi_hand_landmarks:
            for landmark in handLandmarks.landmark:
                x, y, z = landmark.x, landmark.y, landmark.z
                points.append(x)
                points.append(y)
                points.append(z)
            drawingModule.draw_landmarks(frame, handLandmarks, mp_hands.HAND_CONNECTIONS)
        
        return True, np.array(points)
       
    else: 
        return False, 0


In [3]:
tello = Tello()
tello.connect()

print("Temperature: "+str(round((tello.get_temperature() -32 )* 5/9)) + "°C") 
print("Battery: "+ str(tello.get_battery()) + "%")
#
tello.streamon()
frame_read = tello.get_frame_read()


[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'streamon'


Temperature: 19°C
Battery: 27%


[INFO] tello.py - 462 - Response streamon: 'ok'


Nr. 1 (without Threading)

In [4]:
vc = cv2.VideoCapture(0)
cv2.namedWindow("img")

In [97]:
tello.end()
tello.reboot()

[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'ok'
[INFO] tello.py - 471 - Send command (no response expected): 'reboot'


threading doesnt work with cv2.imshow

In [5]:
tello.takeoff()

while True:
    rval, img = vc.read()
    a = np.shape(img)
    drone = frame_read.frame
    drone =  cv2.cvtColor(drone,cv2.COLOR_BGR2RGB)
    drone = cv2.resize(drone, cam_res)
    #img = cv2.resize(img, cam_res)
    plot = np.concatenate((img, drone), axis=1)
    cv2.imshow("img", plot)
    
    bol, res = get_sign(img)
    key = cv2.waitKey(50) & 0xff
    if key == ord(" "): #emergency
        print("emergency")
        tello.emergency()
        break

    elif bol:
        res = np.argmax(model(np.reshape(res, (1, res.shape[0]))))               

        if res == 0: #right
            tello.move_right(10)

        elif res == 1: #left
            tello.move_left(10)

        elif res == 2: # up
            tello.move_up(10)

        elif res == 3: # down
            tello.move_down(10)

        elif res == 4: # fist = vorne
            tello.move_forward(10)

        elif res == 5: # hand = rückwerts
            tello.move_back(10)

        elif res == 6: # V = normal landing
            tello.rotate_clockwise(10)
            #print("normal landing")
          #  break

print("finished")
#tello.land()



[INFO] tello.py - 438 - Send command: 'takeoff'
[INFO] tello.py - 462 - Response takeoff: 'ok'
[INFO] tello.py - 438 - Send command: 'cw 10'
[INFO] tello.py - 462 - Response cw 10: 'ok'
[INFO] tello.py - 438 - Send command: 'forward 10'
[INFO] tello.py - 462 - Response forward 10: 'out of range'
[INFO] tello.py - 438 - Send command: 'forward 10'
[INFO] tello.py - 462 - Response forward 10: 'out of range'
[INFO] tello.py - 438 - Send command: 'forward 10'
[INFO] tello.py - 462 - Response forward 10: 'out of range'


TelloException: Command 'forward 10' was unsuccessful for 4 tries. Latest response:	'out of range'

Nr. 2 (with Threading)

In [8]:
global finished
global img
finished = False
img = False, 0

# distance in cm 20-500

def func():
    d = 20
    while not finished:
        b, frame = img
        if b:
            bol, res = get_sign(frame)

            if bol:
                res = np.argmax(model(np.reshape(res, (1, res.shape[0]))))               

                if res == 0: #right
                    tello.move_right(d)

                elif res == 1: #left
                    tello.move_left(d)

                elif res == 2: # up
                    tello.move_up(d)

                elif res == 3: # down
                    tello.move_down(d)

                elif res == 4: # fist = hoch
                    tello.move_back(d)

                elif res == 5: # hand = runter
                    tello.move_forward(d)

                elif res == 6: # V = rotate
                    tello.rotate_clockwise(30)
                    #print("normal landing")
                #  break
        




In [10]:
tello.takeoff()
t1 = threading.Thread(target=func, daemon=True)
t1.start()
finished = False

while not finished:
    rval, frame = vc.read()
    img = True, frame 
    key = cv2.waitKey(50) & 0xff
    if key == ord(" "):
            print("landing")
            tello.land()
            finished =True
            t.sleep(1)
    elif key == 27: #emergency ESC
                print("emergency")
                tello.emergency()
                finished = True
                t.sleep(1)
    
    drone = frame_read.frame
    drone =  cv2.cvtColor(drone,cv2.COLOR_BGR2RGB)
    drone = cv2.resize(drone, cam_res)
    new = np.concatenate((frame, drone), axis=1)
    cv2.imshow("img", new)

print("finished")



[INFO] tello.py - 438 - Send command: 'takeoff'
[INFO] tello.py - 462 - Response takeoff: 'ok'


KeyboardInterrupt: 

In [11]:
tello.end()
tello.reboot()

[INFO] tello.py - 438 - Send command: 'land'


get stats

In [137]:
D = ((tello.get_current_state()))
df = pd.DataFrame(D, index=[0])
df["templ"] = ((round((df["templ"] -32)*5/9)))
df["temph"] = ((round((df["temph"] -32)*5/9)))
df.head()

,pitch,roll,yaw,vgx,vgy,vgz,templ,temph,tof,h,bat,baro,time,agx,agy,agz
0,0,0,-37,0,0,0,92,36.0,10,0,50,254.83,68,-10.0,9.0,-998.0


In [6]:
tello.land()

[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'ok'


In [13]:
cv2.destroyAllWindows()
vc.release()

options

In [ ]:

elif res == 6: # V = foto
    print("bitte lächeln")
    num_pic += 1
    cv2.imwrite(tello_img + "foto"+ str(num_pic) + ".jpg",drone)

 elif res == 6: # V = normal landing
        print("normal landing")
        tello.land()
        break

elif res == 8:
    tello.flip(direction=)
            

better option

In [ ]:
dist = 30 # distance in cm
def func():
    while not finished:
        b, frame = img
        if b:
            bol, res = get_sign(frame)

            if bol:
                res = np.argmax(model(np.reshape(res, (1, res.shape[0]))))
                if res < 4:
                    res = signs[res] # 0 = right, 1 = left, 2 = up, 3 = down, 4 = fist, 5 = hand, 6 = V !!!! müssten noch unbennet werden in: up, down, left, right, forward or back
                    tello.move(res,dist )#Tello fly up, down, left, right, forward or back with distance x cm
                    
                elif res == 6:
                    tello.land()